# 1M Rows Read/Write

In [9]:
import os
import time
import random
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from chronostore import TimeSeriesEngine, TableSchema, ColumnSchema
from chronostore.backend import FlatFileBackend

# Clean previous test data
os.system("rm -rf ./data")

# Setup Schema, Backend & Engine
# -------------------------------
schema = TableSchema(columns=[
    ColumnSchema("timestamp", "q"), # int64 nanoseconds
    ColumnSchema("price", "d"), # float64
    ColumnSchema("volume", "q"), # int64
])
backend = FlatFileBackend(schema, "./data")
engine = TimeSeriesEngine(backend)


# Write 1 Million Rows
# -------------------------------
n = 1_000_000
symbol = "TestAsset"
date_str = "2025-06-14"
start = datetime(2025, 6, 14, 9, 30)
timestamps = np.array([
    int((start + timedelta(milliseconds=i)).timestamp() * 1e9) for i in range(n)
], dtype=np.int64)
prices = np.random.uniform(4200.0, 4300.0, size=n)
volumes = np.random.randint(1, 10, size=n)

df = pd.DataFrame({
    "timestamp": timestamps,
    "price": prices,
    "volume": volumes,
})

t0 = time.time()
engine.append(symbol, date_str, df)
engine.flush()
write_time = time.time() - t0
print(f"✅ Wrote {n_rows:,} rows in {write_time:.2f}s")


# Read Entire File
# -------------------------------
t1 = time.time()
data = engine.read_dataframe(symbol, date_str)
read_time = time.time() - t1
print(f"✅ Read {len(data['timestamp']):,} rows in {read_time:.2f}s")


# Inspect Sample Output
# -------------------------------
df = pd.DataFrame(data)
display(df.head())
display(df.describe())


# Summary
# -------------------------------
print("\n--- Summary ---")
print(f"Write speed: {n_rows / write_time:,.0f} rows/sec")
print(f"Read speed: {n_rows / read_time:,.0f} rows/sec")

✅ Wrote 1,000,000 rows in 0.04s
✅ Read 1,000,000 rows in 0.02s


,timestamp,price,volume
0,1749907800000000000,4206.726458,9
1,1749907800000999936,4281.597137,7
2,1749907800002000128,4202.638944,6
3,1749907800003000064,4272.176320,7
4,1749907800004000000,4209.450579,2


,timestamp,price,volume
count,1.000000e+06,1000000.000000,1000000.000000
mean,1.749908e+18,4250.002521,4.999807
std,2.886753e+11,28.852402,2.580574
min,1.749908e+18,4200.000081,1.000000
25%,1.749908e+18,4225.050687,3.000000
50%,1.749908e+18,4250.017472,5.000000
75%,1.749909e+18,4274.974897,7.000000
max,1.749909e+18,4299.999867,9.000000



--- Summary ---
Write speed: 27,244,232 rows/sec
Read speed: 61,117,403 rows/sec
